In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, fname) 
                            for fname in os.listdir(root_dir) 
                            if fname.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform
        self.label = label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.label


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset, Dataset
from PIL import Image
import matplotlib.pyplot as plt

# Parameters
batch_size = 128
image_size = 32
nc = 3
ndf = 64
num_epochs = 10
lr = 0.0001  # Lower learning rate for fine-tuning
beta1 = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom dataset class for flat directory structure
class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.image_paths = [
            os.path.join(root_dir, fname)
            for fname in os.listdir(root_dir)
            if fname.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.transform = transform
        self.label = label
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.label

# Image transforms
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Dataset paths
real_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/REAL"
fake_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/FAKE"

# Datasets
real_dataset = FlatImageFolder(real_dir, label=1, transform=transform)
fake_dataset = FlatImageFolder(fake_dir, label=0, transform=transform)

# Combine and create DataLoader
train_dataset = ConcatDataset([real_dataset, fake_dataset])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Discriminator model architecture (kept the same to match the pre-trained model)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.main(x).view(-1)

# Instantiate model
netD = Discriminator().to(device)

# Load the pre-trained discriminator
pretrained_path = "/kaggle/input/discriminatortdl/dcgan_discriminator.pth"
netD.load_state_dict(torch.load(pretrained_path, map_location=device))
print(f"Loaded pre-trained discriminator from {pretrained_path}")

# Loss function and optimizer for fine-tuning
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))

# Fine-tuning loop
def finetune_discriminator():
    D_losses = []
    D_accuracies = []
    print("Fine-tuning Discriminator on CIFAKE dataset...")
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        for i, (data, labels) in enumerate(train_loader):
            # Move data to device
            data = data.to(device)
            labels = labels.float().to(device)
            
            # Forward pass
            netD.zero_grad()
            output = netD(data)
            
            # Calculate loss
            loss = criterion(output, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizerD.step()
            
            # Record loss
            D_losses.append(loss.item())
            epoch_loss += loss.item()
            
            # Calculate accuracy
            predicted = (output >= 0.5).float()
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            # Print progress
            if i % 50 == 0:
                print(f"[{epoch+1}/{num_epochs}][{i}/{len(train_loader)}] Loss_D: {loss.item():.4f}")
        
        # Calculate epoch statistics
        epoch_accuracy = correct_predictions / total_samples
        D_accuracies.append(epoch_accuracy)
        avg_epoch_loss = epoch_loss / len(train_loader)
        
        print(f"Epoch {epoch+1}/{num_epochs} - Avg Loss: {avg_epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
    
    return D_losses, D_accuracies

# Run fine-tuning
D_losses, D_accuracies = finetune_discriminator()

# Plot loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(D_losses)
plt.title("Discriminator Fine-tuning Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.grid(True)

# Plot accuracy per epoch
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), D_accuracies, marker='o')
plt.title("Discriminator Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.tight_layout()

plt.savefig("discriminator_finetuning_cifake.png")
plt.show()

# Save the fine-tuned discriminator
torch.save(netD.state_dict(), "finetuned_discriminator_cifake.pth")

# Evaluate on test set if available
try:
    test_real_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL"
    test_fake_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE"
    
    test_real_dataset = FlatImageFolder(test_real_dir, label=1, transform=transform)
    test_fake_dataset = FlatImageFolder(test_fake_dir, label=0, transform=transform)
    test_dataset = ConcatDataset([test_real_dataset, test_fake_dataset])
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    netD.eval()
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.to(device)
            labels = labels.to(device)
            outputs = netD(data)
            predicted = (outputs >= 0.5).float()
            test_correct += (predicted == labels).sum().item()
            test_total += labels.size(0)
    
    test_accuracy = test_correct / test_total
    print(f"Test accuracy: {test_accuracy:.4f}")
    
except Exception as e:
    print(f"Test set evaluation failed: {e}")
    print("Skipping test set evaluation.")

Loaded pre-trained discriminator from /kaggle/input/discriminatortdl/dcgan_discriminator.pth
Fine-tuning Discriminator on CIFAKE dataset...


/tmp/ipykernel_31/1591142696.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  netD.load_state_dict(torch.load(pretrained_path, map_location=device))


[1/10][0/782] Loss_D: 0.6565


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Parameters
batch_size = 64
image_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 10  # Reduced epochs for fine-tuning
lr_d = 0.00005  # Lower learning rate for fine-tuning
beta1 = 0.5
beta2 = 0.999

# Custom dataset class for flat directory structure
class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.image_paths = [
            os.path.join(root_dir, fname)
            for fname in os.listdir(root_dir)
            if fname.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.transform = transform
        self.label = label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.label

# Image transforms
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Dataset paths
real_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/REAL"
fake_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/FAKE"

# Datasets
real_dataset = FlatImageFolder(real_dir, label=1, transform=transform)
fake_dataset = FlatImageFolder(fake_dir, label=0, transform=transform)

# Combine and create DataLoader
train_dataset = ConcatDataset([real_dataset, fake_dataset])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Equalized learning rate layers
class EqualizedLinear(nn.Module):
    def __init__(self, in_features, out_features, gain=1.0):
        super(EqualizedLinear, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.gain = gain
        
        # Initialize weights using equalized initialization
        self.weight = nn.Parameter(torch.randn(out_features, in_features).div_(gain))
        self.bias = nn.Parameter(torch.zeros(out_features))
        
    def forward(self, x):
        # Apply the equalized weights to the input
        return F.linear(x, self.weight, self.bias)

class EqualizedConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # Initialize weights with scaled variance
        scale = 1 / np.sqrt(in_channels * kernel_size * kernel_size)
        nn.init.normal_(conv.weight.data, 0.0, 1.0)
        conv.weight.data.mul_(scale)
        nn.init.zeros_(conv.bias.data)
        self.conv = conv
        self.scale = scale
        
    def forward(self, x):
        return self.conv(x)

# Fixed MinibatchStdDev implementation
class MinibatchStdDev(nn.Module):
    def __init__(self, group_size=4):
        super().__init__()
        self.group_size = group_size
        
    def forward(self, x):
        batch_size, channels, height, width = x.shape
        
        # If batch size is not divisible by group size, adjust group size
        if batch_size % self.group_size != 0:
            group_size = batch_size
        else:
            group_size = self.group_size
            
        # Ensure group size is not larger than batch size
        group_size = min(group_size, batch_size)
        
        # If group_size is 1, we can't compute std dev across groups, so use a simpler approach
        if group_size == 1:
            # Add a single channel of zeros (simplest fallback)
            zeros = torch.zeros(batch_size, 1, height, width, device=x.device)
            return torch.cat([x, zeros], dim=1)
            
        # Handle group division
        n_groups = batch_size // group_size
        
        # Reshape for group-wise calculation
        y = x.view(group_size, n_groups, channels, height, width)
        
        # Calculate standard deviation for each group
        y = torch.std(y, dim=0, unbiased=False)  # [n_groups, channels, height, width]
        
        # Average over all channels
        y = y.mean(dim=1, keepdim=True)  # [n_groups, 1, height, width]
        
        # Expand to match input batch size
        y = y.repeat(group_size, 1, 1, 1)  # [batch_size, 1, height, width]
        
        # If we don't have exactly batch_size elements, slice the tensor
        if y.size(0) != batch_size:
            y = y[:batch_size]
            
        # Concatenate along channel dimension
        return torch.cat([x, y], dim=1)

# Discriminator
class StyleGANDiscriminator(nn.Module):
    def __init__(self, channels=32):
        super().__init__()
        self.from_rgb = EqualizedConv2d(3, channels, 1)
        self.features = nn.ModuleList([
            nn.Sequential(
                EqualizedConv2d(channels, channels * 2, 4, 2, 1),
                nn.LeakyReLU(0.2)
            ),
            nn.Sequential(
                EqualizedConv2d(channels * 2, channels * 4, 4, 2, 1),
                nn.LeakyReLU(0.2)
            ),
            nn.Sequential(
                EqualizedConv2d(channels * 4, channels * 8, 4, 2, 1),
                nn.LeakyReLU(0.2)
            )
        ])
        self.final_block = nn.Sequential(
            MinibatchStdDev(),
            EqualizedConv2d(channels * 8 + 1, channels * 8, 3, 1, 1),
            nn.LeakyReLU(0.2),
            EqualizedConv2d(channels * 8, channels * 4, 4, 1, 0),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            EqualizedLinear(channels * 4, 1)
        )
        
    def forward(self, x):
        x = self.from_rgb(x)
        for block in self.features:
            x = block(x)
        validity = self.final_block(x)
        return validity

# Initialize Discriminator
discriminator = StyleGANDiscriminator().to(device)

# Load the pre-trained discriminator weights
pretrained_path = "/kaggle/input/discriminatortdl/improved_stylegan_discriminator.pth"
try:
    discriminator.load_state_dict(torch.load(pretrained_path, map_location=device))
    print(f"Successfully loaded pre-trained discriminator from {pretrained_path}")
except Exception as e:
    print(f"Error loading pre-trained model: {e}")
    print("Continuing with randomly initialized weights...")

# Optimizer for discriminator - lower learning rate for fine-tuning
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(beta1, beta2))

# Loss function
def discriminator_loss(real_validity, fake_validity):
    real_loss = F.softplus(-real_validity).mean()
    fake_loss = F.softplus(fake_validity).mean()
    return real_loss + fake_loss

# Function to fine-tune the discriminator
def finetune_discriminator():
    D_losses = []
    D_accuracies = []
    print("Starting Fine-tuning Loop for Discriminator...")
    
    for epoch in range(num_epochs):
        total_d_loss = 0
        num_batches = 0
        correct_predictions = 0
        total_samples = 0
        
        for batch_imgs, batch_labels in train_loader:
            batch_size = batch_imgs.size(0)
            
            # Separate real and fake images based on labels
            real_mask = (batch_labels == 1)
            fake_mask = (batch_labels == 0)
            
            real_imgs = batch_imgs[real_mask].to(device)
            fake_imgs = batch_imgs[fake_mask].to(device)
            
            # Skip this batch if either all real or all fake
            if len(real_imgs) == 0 or len(fake_imgs) == 0:
                continue
                
            # Make sure we have equal numbers of real and fake images
            min_size = min(len(real_imgs), len(fake_imgs))
            real_imgs = real_imgs[:min_size]
            fake_imgs = fake_imgs[:min_size]
            
            # Combine for accuracy calculation
            all_imgs = torch.cat([real_imgs, fake_imgs], dim=0)
            all_labels = torch.cat([
                torch.ones(min_size, device=device),
                torch.zeros(min_size, device=device)
            ])
            
            # Forward pass through discriminator
            d_optimizer.zero_grad()
            
            real_validity = discriminator(real_imgs)
            fake_validity = discriminator(fake_imgs)
            
            # Calculate accuracy
            all_validity = discriminator(all_imgs)
            predicted_labels = (all_validity > 0).float().view(-1)
            correct = (predicted_labels == all_labels).sum().item()
            correct_predictions += correct
            total_samples += len(all_labels)
            
            # Compute loss for discriminator
            d_loss = discriminator_loss(real_validity, fake_validity)
            
            # Backpropagation and optimization
            d_loss.backward()
            d_optimizer.step()
            
            total_d_loss += d_loss.item()
            num_batches += 1
            
            # Save loss for plotting
            D_losses.append(d_loss.item())
            
            # Print progress
            if num_batches % 50 == 0:
                print(f"[Epoch {epoch+1}/{num_epochs}] [Batch {num_batches}/{len(train_loader)}] "
                      f"[D loss: {d_loss.item():.4f}]")
        
        # Calculate accuracy for this epoch
        epoch_accuracy = correct_predictions / total_samples if total_samples > 0 else 0
        D_accuracies.append(epoch_accuracy)
        
        # Print epoch summary
        avg_loss = total_d_loss / num_batches if num_batches > 0 else 0
        print(f"Epoch [{epoch+1}/{num_epochs}], Discriminator Loss: {avg_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
    
    return D_losses, D_accuracies

# Run fine-tuning for the discriminator
D_losses, D_accuracies = finetune_discriminator()

# Plot discriminator loss and accuracy
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(D_losses)
plt.title("Discriminator Fine-tuning Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.grid(True)

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), D_accuracies, marker='o')
plt.title("Discriminator Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)

# Save the plot to a file
plt.tight_layout()
plt.savefig("discriminator_finetuning_stylegan_cifake.png")

# Display the plot
plt.show()

# Save the fine-tuned discriminator model
torch.save(discriminator.state_dict(), "finetuned_stylegan_discriminator_cifake.pth")

# Evaluate on test set if available
try:
    test_real_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL"
    test_fake_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE"
    
    test_real_dataset = FlatImageFolder(test_real_dir, label=1, transform=transform)
    test_fake_dataset = FlatImageFolder(test_fake_dir, label=0, transform=transform)
    test_dataset = ConcatDataset([test_real_dataset, test_fake_dataset])
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    discriminator.eval()
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for batch_imgs, batch_labels in test_loader:
            batch_imgs = batch_imgs.to(device)
            batch_labels = batch_labels.float().to(device)
            
            outputs = discriminator(batch_imgs)
            predicted = (outputs > 0).float().view(-1)
            test_correct += (predicted == batch_labels).sum().item()
            test_total += batch_labels.size(0)
    
    test_accuracy = test_correct / test_total
    print(f"Test accuracy: {test_accuracy:.4f}")
    
except Exception as e:
    print(f"Test set evaluation failed: {e}")
    print("Skipping test set evaluation.")

print("Fine-tuning complete! Discriminator saved to 'finetuned_stylegan_discriminator_cifake.pth'")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torchvision.utils import save_image, make_grid
import os
from PIL import Image

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Parameters
batch_size = 64
img_channels = 3
image_size = 32
num_epochs = 10
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define transforms
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Custom Dataset for loading flat directory of images with a specific label
class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.label = label
        self.image_files = [os.path.join(root_dir, f) for f in os.listdir(root_dir)
                            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.label

# Dataset paths
real_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/REAL"
fake_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/FAKE"

# Datasets
real_dataset = FlatImageFolder(real_dir, label=1, transform=transform)
fake_dataset = FlatImageFolder(fake_dir, label=0, transform=transform)

# Combine and create DataLoader
train_dataset = ConcatDataset([real_dataset, fake_dataset])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Define the Discriminator model from IC-GAN
class Discriminator(nn.Module):
    def __init__(self, img_channels=3):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(img_channels, 32, 3, 2, 1),  # 32x32 -> 16x16
            nn.LeakyReLU(0.2),

            nn.Conv2d(32, 64, 3, 2, 1),  # 16x16 -> 8x8
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, 3, 2, 1),  # 8x8 -> 4x4
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )

        self.final = nn.Conv2d(128, 256, 4, 1, 0)  # 4x4 -> 1x1
        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        features = self.features(img)
        final_features = self.final(features)
        validity = self.output(final_features)
        return validity, final_features

# Discriminator Training Class with Pretrained Weights
class DiscriminatorTrainer:
    def __init__(self):
        self.discriminator = Discriminator(img_channels).to(device)

        # Load pretrained weights from IC-GAN
        pretrained_path = "/kaggle/input/discriminatortdl/icgan_discriminator.pth"
        state_dict = torch.load(pretrained_path, map_location=device)
        self.discriminator.load_state_dict(state_dict, strict=False)
        print(f"✅ Loaded pretrained weights from {pretrained_path}")

        self.optimizer = optim.Adam(self.discriminator.parameters(), lr=lr, betas=(beta1, beta2))
        self.criterion = nn.BCELoss()
        self.model_name = "CIFAKE-Discriminator-Finetune"
        self.losses = []
        self.accuracies = []

    def train(self, dataloader, num_epochs):
        print(f"🚀 Starting {self.model_name} Fine-tuning...")

        for epoch in range(num_epochs):
            epoch_loss = 0
            correct = 0
            total = 0

            for i, (images, labels) in enumerate(dataloader):
                images = images.to(device)
                labels = labels.float().to(device).view(-1, 1)

                self.optimizer.zero_grad()
                predictions, _ = self.discriminator(images)
                loss = self.criterion(predictions, labels)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()
                predicted = (predictions >= 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

                if i % 50 == 0:
                    print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] "
                          f"[D loss: {loss.item():.4f}]")

            epoch_accuracy = 100 * correct / total
            self.losses.append(epoch_loss / len(dataloader))
            self.accuracies.append(epoch_accuracy)
            print(f"📈 Epoch {epoch} - Loss: {epoch_loss/len(dataloader):.4f}, Accuracy: {epoch_accuracy:.2f}%")

        torch.save(self.discriminator.state_dict(), "cifake_discriminator_icgan_finetuned.pth")
        return self.losses, self.accuracies

    def evaluate(self, dataloader):
        self.discriminator.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in dataloader:
                images = images.to(device)
                labels = labels.float().to(device).view(-1, 1)
                predictions, _ = self.discriminator(images)
                predicted = (predictions >= 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        accuracy = 100 * correct / total
        print(f"✅ Evaluation accuracy: {accuracy:.2f}%")
        return accuracy

    def plot_metrics(self):
        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        plt.plot(self.losses)
        plt.title("Discriminator Loss During Fine-tuning")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")

        plt.subplot(1, 2, 2)
        plt.plot(self.accuracies)
        plt.title("Discriminator Accuracy During Fine-tuning")
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy (%)")

        plt.tight_layout()
        plt.savefig("finetune_metrics_icgan.png")
        plt.show()

    def visualize_samples(self, dataloader, num_samples=8):
        self.discriminator.eval()
        images, labels = next(iter(dataloader))
        images = images[:num_samples].to(device)
        labels = labels[:num_samples].float().to(device).view(-1, 1)

        with torch.no_grad():
            predictions, _ = self.discriminator(images)
            predicted_labels = (predictions >= 0.5).float()
            images = (images.cpu() + 1) / 2  # Denormalize

            fig, axes = plt.subplots(1, num_samples, figsize=(16, 4))
            for i in range(num_samples):
                axes[i].imshow(images[i].permute(1, 2, 0))
                real_label = "Real" if labels[i].item() == 1 else "Fake"
                pred_label = "Real" if predicted_labels[i].item() == 1 else "Fake"
                color = "green" if labels[i].item() == predicted_labels[i].item() else "red"
                axes[i].set_title(f"True: {real_label}\nPred: {pred_label}", color=color)
                axes[i].axis('off')

            plt.tight_layout()
            plt.savefig("sample_predictions_finetune_icgan.png")
            plt.show()

# Run everything
trainer = DiscriminatorTrainer()
losses, accuracies = trainer.train(train_loader, num_epochs)
trainer.plot_metrics()
trainer.visualize_samples(train_loader)
print("✅ CIFAKE Discriminator fine-tuning complete!")


# EVALUATION

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torchvision.utils import save_image, make_grid
import os
from PIL import Image

# 1. Define your Discriminator class (already done by you)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.main(x).view(-1)


In [3]:

batch_size = 64
image_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.image_paths = [
            os.path.join(root_dir, fname)
            for fname in os.listdir(root_dir)
            if fname.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.transform = transform
        self.label = label
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.label

# Image transforms
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# STYLEGAN

In [23]:
# ── 1) Imports & Seed ─────────────────────────────────────────────────────────
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# ── 2) Custom Layers (with FIXED MinibatchStdDev) ─────────────────────────────
class EqualizedLinear(nn.Module):
    def __init__(self, in_f, out_f, gain=1.0):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_f, in_f).div_(gain))
        self.bias   = nn.Parameter(torch.zeros(out_f))
    def forward(self, x):
        return F.linear(x, self.weight, self.bias)

class EqualizedConv2d(nn.Module):
    def __init__(self, in_ch, out_ch, k, stride=1, pad=0):
        super().__init__()
        conv = nn.Conv2d(in_ch, out_ch, k, stride, pad)
        scale = 1/np.sqrt(in_ch * k * k)
        nn.init.normal_(conv.weight, 0.0, 1.0)
        conv.weight.data.mul_(scale)
        nn.init.zeros_(conv.bias)
        self.conv = conv
    def forward(self, x):
        return self.conv(x)

class MinibatchStdDev(nn.Module):
    def __init__(self, group_size=4):
        super().__init__()
        self.group_size = group_size

    def forward(self, x):
        bs, ch, h, w = x.shape
        # pick group_size <= bs
        g = min(self.group_size, bs)
        if g <= 1:
            zeros = torch.zeros(bs, 1, h, w, device=x.device)
            return torch.cat([x, zeros], dim=1)

        # reshape into (g, n_groups, ch, h, w)
        n = bs // g
        y = x.view(g, n, ch, h, w)
        y = y.std(dim=0, unbiased=False)        # → (n, ch, h, w)
        y = y.mean(dim=1, keepdim=True)         # → (n, 1, h, w)
        # expand only along the batch axis, keep h,w intact
        y = y.repeat(g, 1, 1, 1)                # → (n*g=bs, 1, h, w)
        y = y[:bs]                              # just in case
        return torch.cat([x, y], dim=1)         # → (bs, ch+1, h, w)


# ── 3) StyleGAN Discriminator Definition ────────────────────────────────────────
class StyleGANDiscriminator(nn.Module):
    def __init__(self, channels=32):
        super().__init__()
        self.from_rgb = EqualizedConv2d(3, channels, 1)
        self.features = nn.ModuleList([
            nn.Sequential(EqualizedConv2d(channels,   channels*2, 4, 2, 1),
                          nn.LeakyReLU(0.2)),
            nn.Sequential(EqualizedConv2d(channels*2, channels*4, 4, 2, 1),
                          nn.LeakyReLU(0.2)),
            nn.Sequential(EqualizedConv2d(channels*4, channels*8, 4, 2, 1),
                          nn.LeakyReLU(0.2)),
        ])
        self.final_block = nn.Sequential(
            MinibatchStdDev(),
            EqualizedConv2d(channels*8 + 1, channels*8, 3, 1, 1),
            nn.LeakyReLU(0.2),
            EqualizedConv2d(channels*8, channels*4, 4, 1, 0),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            EqualizedLinear(channels*4, 1)
        )

    def forward(self, x):
        x = self.from_rgb(x)
        for block in self.features:
            x = block(x)
        return self.final_block(x).view(-1)   # → (batch_size,)


# ── 4) Build Test DataLoader ───────────────────────────────────────────────────
class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.paths = sorted(
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith(('.png','.jpg','jpeg'))
        )
        self.transform = transform
        self.label = label

    def __len__(self): return len(self.paths)
    def __getitem__(self, i):
        img = Image.open(self.paths[i]).convert('RGB')
        if self.transform: img = self.transform(img)
        return img, self.label

image_size = 32
batch_size = 64
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

real_test = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL"
fake_test = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE"

real_ds = FlatImageFolder(real_test, 1, transform)
fake_ds = FlatImageFolder(fake_test, 0, transform)
test_loader = DataLoader(
    ConcatDataset([real_ds, fake_ds]),
    batch_size=batch_size, shuffle=False, num_workers=2
)


# ── 5) Instantiate & Load with Key Remapping ─────────────────────────────────
model = StyleGANDiscriminator().to(device)
ckpt = "/kaggle/input/finetunedgans/finetuned_stylegan_discriminator_cifake.pth"
raw_state = torch.load(ckpt, map_location=device)

fixed_state = {}
for k,v in raw_state.items():
    if k.startswith("final_block.6.linear"):
        fixed_state[k.replace("final_block.6.linear", "final_block.6")] = v
    else:
        fixed_state[k] = v

# load and report any leftover mismatches
miss, unexp = model.load_state_dict(fixed_state, strict=False)
print("Missing keys:", miss)
print("Unexpected keys:", unexp)
model.eval()


# ── 6) Evaluation Function & Run ───────────────────────────────────────────────
def evaluate_discriminator(model, loader, name="Discriminator"):
    all_p, all_preds, all_labels = [], [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            logits = model(imgs)          # already linear outputs
            probs  = torch.sigmoid(logits)
            preds  = (probs >= 0.5).long()

            all_p.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels)

    acc = accuracy_score(all_labels, all_preds)
    cm  = confusion_matrix(all_labels, all_preds)
    cr  = classification_report(all_labels, all_preds, target_names=["Fake","Real"])
    auc = roc_auc_score(all_labels, all_p)

    print(f"\n🧪 Evaluation for {name}")
    print(f"✅ Accuracy: {acc * 100:.2f}%")
    print(f"📊 Confusion Matrix:\n{cm}")
    print(f"📋 Classification Report:\n{cr}")
    print(f"📈 ROC-AUC Score: {auc:.4f}")

# finally, run it:
evaluate_discriminator(model, test_loader, name="StyleGAN Discriminator")


Using device: cuda


/tmp/ipykernel_31/488887005.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  raw_state = torch.load(ckpt, map_location=device)


Missing keys: []
Unexpected keys: []

🧪 Evaluation for StyleGAN Discriminator
✅ Accuracy: 96.89%
📊 Confusion Matrix:
[[9531  469]
 [ 153 9847]]
📋 Classification Report:
              precision    recall  f1-score   support

        Fake       0.98      0.95      0.97     10000
        Real       0.95      0.98      0.97     10000

    accuracy                           0.97     20000
   macro avg       0.97      0.97      0.97     20000
weighted avg       0.97      0.97      0.97     20000

📈 ROC-AUC Score: 0.9959


# DCGAN


In [24]:
# Notebook: Load and Evaluate Fine-tuned DCGAN Discriminator

# ── 1) Imports & Seed ─────────────────────────────────────────────────────────
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# Reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ── 2) Dataset Class ──────────────────────────────────────────────────────────
class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.paths = sorted(
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        )
        self.transform = transform
        self.label = label

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.label

# ── 3) Transforms & Test DataLoader ───────────────────────────────────────────
image_size = 32
batch_size = 128
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),  # normalize RGB
])

real_test_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL"
fake_test_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE"

real_test_ds = FlatImageFolder(real_test_dir, label=1, transform=transform)
fake_test_ds = FlatImageFolder(fake_test_dir, label=0, transform=transform)

test_loader = DataLoader(
    ConcatDataset([real_test_ds, fake_test_ds]),
    batch_size=batch_size, shuffle=False, num_workers=2
)

# ── 4) Discriminator Definition ───────────────────────────────────────────────
nc, ndf = 3, 64  # channels, feature maps

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1)

# ── 5) Instantiate & Load Fine-tuned Model ────────────────────────────────────
model = Discriminator().to(device)
ckpt_path = "/kaggle/input/finetunedgans/finetuned_discriminator_cifake.pth"  # adjust path if needed
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state)
model.eval()
print(f"Loaded fine-tuned discriminator from {ckpt_path}")

# ── 6) Evaluation Function ────────────────────────────────────────────────────
def evaluate_discriminator(model, loader, name="Discriminator"):
    model.eval()
    all_probs, all_preds, all_labels = [], [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            # Forward pass
            outputs = model(imgs)  # already sigmoid-activated
            probs = outputs.cpu().numpy()
            preds = (probs >= 0.5).astype(int)

            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels)

    # Metrics
    acc = accuracy_score(all_labels, all_preds)
    cm  = confusion_matrix(all_labels, all_preds)
    cr  = classification_report(all_labels, all_preds, target_names=["Fake","Real"])
    auc = roc_auc_score(all_labels, all_probs)

    # Print results
    print(f"\n🧪 Evaluation for {name}")
    print(f"✅ Accuracy: {acc * 100:.2f}%")
    print(f"📊 Confusion Matrix:\n{cm}")
    print(f"📋 Classification Report:\n{cr}")
    print(f"📈 ROC-AUC Score: {auc:.4f}")

# ── 7) Run Evaluation ─────────────────────────────────────────────────────────
evaluate_discriminator(model, test_loader, name="DCGAN Discriminator")


Using device: cuda
Loaded fine-tuned discriminator from /kaggle/input/finetunedgans/finetuned_discriminator_cifake.pth


/tmp/ipykernel_31/1969671936.py:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location=device)



🧪 Evaluation for DCGAN Discriminator
✅ Accuracy: 94.71%
📊 Confusion Matrix:
[[9533  467]
 [ 591 9409]]
📋 Classification Report:
              precision    recall  f1-score   support

        Fake       0.94      0.95      0.95     10000
        Real       0.95      0.94      0.95     10000

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000

📈 ROC-AUC Score: 0.9883


# ICGAN

In [26]:
# Notebook: Load and Evaluate Fine-tuned IC-GAN Discriminator

# ── 1) Imports & Seed ─────────────────────────────────────────────────────────
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# Reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ── 2) Dataset & Transforms ────────────────────────────────────────────────────
class FlatImageFolder(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.image_paths = sorted(
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith(('.png','.jpg','.jpeg'))
        )
        self.transform = transform
        self.label = label

    def __len__(self): return len(self.image_paths)
    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.label

# Parameters
batch_size  = 64
image_size  = 32
img_channels = 3

# Transforms
tf = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

# Test directories (adjust paths as needed)
real_test_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL"
fake_test_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE"

real_ds = FlatImageFolder(real_test_dir, 1, tf)
fake_ds = FlatImageFolder(fake_test_dir, 0, tf)

test_loader = DataLoader(
    ConcatDataset([real_ds, fake_ds]),
    batch_size=batch_size, shuffle=False, num_workers=2
)

# ── 3) Model Definition ─────────────────────────────────────────────────────────
class Discriminator(nn.Module):
    def __init__(self, img_channels=3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(img_channels, 32, 3, 2, 1),  # 32x32 -> 16x16
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 3, 2, 1),             # 16x16 -> 8x8
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, 2, 1),            # 8x8 -> 4x4
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )
        self.final = nn.Conv2d(128, 256, 4, 1, 0)   # 4x4 -> 1x1
        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x_feat = self.features(x)
        x_fin  = self.final(x_feat)
        validity = self.output(x_fin)
        return validity.view(-1)

# ── 4) Instantiate & Load Checkpoint ─────────────────────────────────────────────
model = Discriminator(img_channels).to(device)
ckpt_path = "/kaggle/input/finetunedgans/cifake_discriminator_icgan_finetuned.pth"  # adjust if needed
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state)
model.eval()
print(f"Loaded fine-tuned IC-GAN Discriminator from {ckpt_path}")

# ── 5) Evaluation Function ─────────────────────────────────────────────────────
def evaluate_discriminator(model, loader, name="Discriminator"):
    all_probs, all_preds, all_labels = [], [], []
    model.eval()
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            probs = model(imgs).cpu().numpy()       # outputs in [0,1]
            preds = (probs >= 0.5).astype(int)
            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels)

    acc = accuracy_score(all_labels, all_preds)
    cm  = confusion_matrix(all_labels, all_preds)
    cr  = classification_report(all_labels, all_preds, target_names=["Fake","Real"])
    auc = roc_auc_score(all_labels, all_probs)

    print(f"\n🧪 Evaluation for {name}")
    print(f"✅ Accuracy: {acc * 100:.2f}%")
    print(f"📊 Confusion Matrix:\n{cm}")
    print(f"📋 Classification Report:\n{cr}")
    print(f"📈 ROC-AUC Score: {auc:.4f}")

# ── 6) Run Evaluation ───────────────────────────────────────────────────────────
evaluate_discriminator(model, test_loader, name="IC-GAN Discriminator")


Using device: cuda
Loaded fine-tuned IC-GAN Discriminator from /kaggle/input/finetunedgans/cifake_discriminator_icgan_finetuned.pth


/tmp/ipykernel_31/2317630900.py:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location=device)



🧪 Evaluation for IC-GAN Discriminator
✅ Accuracy: 94.19%
📊 Confusion Matrix:
[[9419  581]
 [ 582 9418]]
📋 Classification Report:
              precision    recall  f1-score   support

        Fake       0.94      0.94      0.94     10000
        Real       0.94      0.94      0.94     10000

    accuracy                           0.94     20000
   macro avg       0.94      0.94      0.94     20000
weighted avg       0.94      0.94      0.94     20000

📈 ROC-AUC Score: 0.9858
